In [4]:
# https://www.youtube.com/watch?v=QEaBAZQCtwE
!pip install transformers

import torch
import torch.nn.functional as F
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 109.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 41.7 MB/s eta 0:00:00


In [5]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [6]:
X_train = ["I've been waiting for a HuggingFace course my whole life.",
           "Python is great!"]

res = classifier(X_train)
res

[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'POSITIVE', 'score': 0.9998615980148315}]

In [7]:
# this does the same thing as the code above
# look at the last 2 numbers on the tensors on the results of this code
# 9.5980e-01 and 9.9986e-01 is the same as above
batch = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors="pt")
batch

# inference
with torch.no_grad():
  # ** means unpack the batch dictionary
  outputs = model(**batch)
  predictions = F.softmax(outputs.logits, dim=1)
  labels = torch.argmax(predictions, dim=1)
  print(outputs, predictions, labels)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [-4.2745,  4.6111]]), hidden_states=None, attentions=None) tensor([[4.0195e-02, 9.5980e-01],
        [1.3835e-04, 9.9986e-01]]) tensor([1, 1])


In [8]:
# save your model
save_directory = "saved"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

# load the model you saved
tok = AutoTokenizer.from_pretrained(save_directory)
mod = AutoModelForSequenceClassification.from_pretrained(save_directory)